# NYC-Flights

## 01 - Problema (caso de estudio)

### Resumen

NYC-Flights (vuelos en la ciudad de Nueva York)

**Objectivos:**
+ Conectar Python con el servidor de SQL
+ Usar queries SQL para responder las siguientes preguntas 



**Preguntas:**:  

1. ¿Cuantos vuelos hubo desde NYC a Seattle en 1993?
2. ¿Cuantas aerolineas vuelan desde NYC a Seattle?
3. ¿Cuantos aviones diferentes haan volado desde NYC a Seattle?
4. ¿Cual es la media del retraso de llegada para los vuelos entre NYC y Seattle?
5. ¿Cual es la proporcion de vuelos a Seattle para cada aeropuerto en NYC?
6. ¿Cual fue la fecha con el mayor retraso de salida?
7. ¿Cual fue la fecha con el mayor retraso de llegada?
8. ¿Cual fue el peor dia para volar desde NYC en 1997 si no te gustan los retrasos?
9. ¿Hay patrones estacionales en los retrasos de salida de los vuelos desde NYC?
10. De media, ¿como varian los retrasos de salida a lo largo del año?
11. ¿Cual fue el vuelo desde NYC mas rapido en la ultima decada?
12. ¿Cuales son los vuelos diarios?¿A donde vuelan?  (vuelo = aerolinea(carrier) + vuelo(flight) + destino(dest)) 
13. ¿Que aerolinea es la que mejor funciono y cual peor en 1999? (pensando en el delay)

## 02 - Obteniendo datos

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('data-nyc/1990.csv')

data.head()

In [ ]:
data.info(memory_usage='deep')

In [ ]:
data=data.fillna('null')

In [ ]:
data.head()

**de pandas a SQL**

### Conexion Python-SQL

In [ ]:
import mysql.connector  # conexion con sql

In [ ]:
db_name='Flights_NYC'           # nombre base de datos

In [ ]:
# crear conexion
create_db=mysql.connector.connect(host='localhost', user='root', passwd='password')

cursor=create_db.cursor()   

In [ ]:
# eliminar base de datos (si existe) y crear un base de datos vacia

cursor.execute('drop database if exists {}'.format(db_name))
cursor.execute('create database {}'.format(db_name))

In [ ]:
# comprueba, muestra las bases de datos 

cursor.execute('show databases')
for x in cursor:
  print(x)

### Cargar datos en SQL

In [ ]:
# conexion a la base de datos

db=mysql.connector.connect(host='localhost', user='root', passwd='password', database=db_name)

cursor=db.cursor()

In [ ]:
# crear tabla en la base de datos

import re     # regex

table_name='_1990'     

cursor.execute('use {};'.format(db_name))

cursor.execute('drop table if exists {};'.format(table_name)) 

# nombres de columna y tipo de dato de la tabla desde el dataframe
names_dtypes=[' '.join(f) for f in zip(data.columns,
                                       [re.findall('[a-t]+',str(e))[0] if e!='object' else 'text' for e in data.dtypes.tolist()])]

table='create table {}({});'.format(table_name, ', '.join(names_dtypes))

table

In [ ]:
cursor.execute(table) # ejecuta la query

In [ ]:
# comprueba las tablas

cursor.execute('show tables')
for x in cursor:
  print(x)

In [ ]:
%%time
results=list(data.T.to_dict().values())  # cambio de formato

In [ ]:
%%time
for i in range(len(results)):  # insert query
    
    insert_query='insert into {} ({}) values {};'\
                  .format(table_name, ','.join(results[i].keys()), tuple(results[i].values()))
    cursor.execute(insert_query)
    
    
db.commit()  # guardar base de datos

In [ ]:
insert_query  # ultima insercion

In [ ]:
# comprobacion

cursor.execute('select * from {} limit 3'.format(table_name))
for x in cursor:
  print(x)

In [ ]:
# todo junto

def to_sql(year):
    
    table_name='_199{}'.format(year)
    data=pd.read_csv('data-nyc/199{}.csv'.format(year)).fillna('null')
         
    cursor.execute('drop table if exists {};'.format(table_name)) 
    names_dtypes=[' '.join(f) for f in zip(data.columns,
                                           [re.findall('[a-t]+',str(e))[0] if e!='object' else 'text' for e in data.dtypes.tolist()])]

    table='create table {}({});'.format(table_name, ', '.join(names_dtypes))
    cursor.execute(table)

    
    results=list(data.T.to_dict().values())
    
    for i in range(len(results)):

        insert_query='insert into {} ({}) values {};'\
                      .format(table_name, ','.join(results[i].keys()), tuple(results[i].values()))
        cursor.execute(insert_query)


    db.commit()

In [ ]:
%%time
from tqdm import tqdm

for i in tqdm(range(10), desc='Data to SQL-->'):
    to_sql(i)

In [ ]:
# comprobacion tablas

cursor.execute('show tables')
for x in cursor:
  print(x)

### Datos desde SQL

In [ ]:
def from_sql(cursor, query):
    print ('Query:\n{}\n'.format(query))
    
    cursor.execute(query)
    data=cursor.fetchall()

    df=pd.DataFrame(data, columns=cursor.column_names)

    print ('Datos leidos desde MySQL.')

    return df

In [ ]:
# reinicia conexion con base de datos

db_name='Flights_NYC'

db=mysql.connector.connect(host='localhost', user='root', passwd='password', database=db_name)

cursor=db.cursor()

# Queries

### Una Tabla

In [ ]:
%%time

query='select * from _1990'

from_sql(cursor, query)

In [ ]:
%%time

query='''select * 
          from 
          information_schema.columns 
          where table_name='_1999';'''


from_sql(cursor, query)

### Todos los datos

In [ ]:
%%time

query='''select * from
         information_schema.tables
            where table_schema='Flights_NYC';'''


from_sql(cursor, query)

In [ ]:
%%time

query='''select * from _1990
         union all
         select * from _1992;'''


from_sql(cursor, query)

In [ ]:
%%time

query='select * from _1990 '+\
      ' '.join(['union all select * from _199{}'.format(i+1) for i in range(3)])+';'


from_sql(cursor, query)

### Limpieza de datos

In [ ]:
%%time

query='''
        select * from
        _1990;
'''


from_sql(cursor, query).shape

In [ ]:
%%time

query='''
        select * from
        _1990
        where DepTime is not null and
              ArrTime is not null;

'''


from_sql(cursor, query)

In [ ]:
%%time

query='''
        select * from
        _1990
        where (DepTime!='null') and 
              (ArrTime!='null') ;

'''


from_sql(cursor, query)

In [ ]:
%%time

all_time_query='select * from _1990 '+\
               ' '.join(['union all select * from _199{}'.format(i+1) for i in range(9)])


clean_query='''
            select * from
            ({}) t
            where (t.DepTime!='null') and 
                  (t.ArrTime!='null') and
                  (t.TailNum!='null')

    '''.format(all_time_query)



from_sql(cursor, clean_query)

## 03 - Preguntas

### 1. ¿Cuantos vuelos hubo desde NYC a Seattle en 1993?


In [ ]:
%%time

query='''
        select count(*) as Flights from
        _1993
        where (Dest='SEA');

'''


from_sql(cursor, query)

### 2. ¿Cuantas aerolineas vuelan desde NYC a Seattle?

In [ ]:
%%time

query='''
        select UniqueCarrier as Airline, count(UniqueCarrier) as Flights
        from _1999
        group by UniqueCarrier;

'''

from_sql(cursor, query)

### 3. ¿Cuantos aviones diferentes haan volado desde NYC a Seattle?

In [ ]:
%%time

query='''
        select c.TailNum , count(c.TailNum) as Flights
        from
        ({}) c 
        group by c.TailNum;
        
'''.format(clean_query)


from_sql(cursor, query)

### 4. ¿Cual es la media del retraso de llegada para los vuelos entre NYC y Seattle?

In [ ]:
%%time

query='''
        select avg(a.ArrDelay) as AvgDelay
        from
        ({}) a
        where(a.Dest='SEA');

'''.format(all_time_query)


from_sql(cursor, query)

### 5. ¿Cual es la proporcion de vuelos a Seattle para cada aeropuerto en NYC?

In [ ]:
%%time

query='''
        select Origin , 
        (count(Origin)*100/(select count(*) from _1999 where (Dest='SEA'))) as PropFlights
        from 
        _1999
        where (Dest='SEA')
        group by Origin;

'''


from_sql(cursor, query)

In [ ]:
%%time

query='''
        select a.Origin , 
        (count(a.Origin)*100/(select count(*) from ({}) as p where (Dest='SEA'))) as PropFlights
        from 
        ({}) a
        where (a.Dest='SEA')
        group by a.Origin;

'''.format(all_time_query, all_time_query)


from_sql(cursor, query)

### 6. ¿Cual fue la fecha con el mayor retraso de salida?

In [1]:
%%time

query='''
        select a.DayofMonth as Day, a.Month, a.Year, avg(a.DepDelay) as DepDelay
        from ({}) a
        group by a.DayofMonth, a.Month, a.Year
        order by DepDelay desc 
        limit 10;

'''.format(clean_query)


from_sql(cursor, query)

NameError: name 'clean_query' is not defined

### 7. ¿Cual fue la fecha con el mayor retraso de llegada?

In [ ]:
%%time

query='''
        select a.DayofMonth as Day, a.Month, a.Year, avg(a.ArrDelay) as ArrDelay
        from ({}) a
        group by a.DayofMonth, a.Month, a.Year
        order by ArrDelay desc 
        limit 10;

'''.format(clean_query)


from_sql(cursor, query)

### 8. ¿Cual fue el peor dia para volar desde NYC en 1997 si no te gustan los retrasos?

In [ ]:
%%time

query='''
        select DayofMonth as Day, Month, Year, avg(DepDelay) as DepDelay
        from _1997
        group by DayofMonth, Month, Year
        order by DepDelay desc 
        limit 10;

'''


from_sql(cursor, query)

### 9. ¿Hay patrones estacionales en los retrasos de salida de los vuelos desde NYC?

In [ ]:
%%time

query='''
        select Month, Year, avg(DepDelay) as DepDelay
        from _1999
        group by Month, Year;

'''


from_sql(cursor, query)

In [ ]:
%%time
%matplotlib inline

query='''
        select Month, Year, avg(DepDelay) as DepDelay
        from _1999
        group by Month, Year;

'''


from_sql(cursor, query).plot(x='Month', y='DepDelay');

In [ ]:
%%time

query='''
        select a.Month, a.Year, avg(a.DepDelay) as DepDelay
        from
        ({}) a
        group by a.Month, a.Year;

'''.format(all_time_query)


from_sql(cursor, query)

### 10. De media, ¿como varian los retrasos de salida a lo largo del año?

In [ ]:
%%time

query='''
        select Month, avg(DepTime) as AvgDepDelay
        from _1999
        group by Month;

'''

from_sql(cursor, query)

### 11. ¿Cual fue el vuelo desde NYC mas rapido en la ultima decada?

In [ ]:
%%time

query='''
        select a.DayofMonth, a.Month, a.Year, a.Distance/a.AirTime as Speed, a.UniqueCarrier as Airline, a.TailNum
        from
        ({}) a
        where (a.Distance!='null') and (a.AirTime!='null')
        order by Speed desc
        limit 5;

'''.format(all_time_query)


from_sql(cursor, query)

### 12. ¿Cuales son los vuelos diarios?¿A donde vuelan?  

In [ ]:
%%time

query='''
        select UniqueCarrier as Airline, FlightNum, Dest, count(Dest) as Flights
        from
        _1999
        group by UniqueCarrier, FlightNum, Dest
        order by Flights desc
        limit 10;

'''


from_sql(cursor, query)

### 13. ¿Que aerolinea es la que mejor funciono y cual peor en 1999?

In [ ]:
%%time

query='''
        select UniqueCarrier as Airline, avg(DepTime) as AvgDepDelay
        from
        _1999
        group by Airline
        order by AvgDepDelay asc;

'''


from_sql(cursor, query)

In [ ]:
%%time

query='''
        select UniqueCarrier as Airline, avg(ArrTime) as AvgArrDelay
        from
        _1999
        group by Airline
        order by AvgArrDelay asc;

'''


from_sql(cursor, query)

In [ ]:
%%time

query='''
        select a.UniqueCarrier as Airline, avg(a.DepTime) as AvgDepDelay
        from
        ({}) a
        group by Airline
        order by AvgDepDelay asc;

'''.format(all_time_query)


from_sql(cursor, query)